In [20]:
from processing import convert_to_mne
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')

path = 'recordings/recording_6/take_2/playing/playing_tk2_rec6_07_05_2024_10_48_41.csv'

rescale = 1e6
fs = 250
chs = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8"]
columns = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8", "AccX", "AccY", "AccZ", "GyroX", "GyroY", "GyroZ",
           "CNT", "BAT", "VALID", "DeltaTime", "Trigger"]

if 'recordings' in path:
    delimiter = ','
else:
    delimiter = '\t'

data = pd.read_csv(filepath_or_buffer=path, delimiter=',')
data.columns = columns[:data.shape[1]-1] + [columns[-1]]

print(data.head())

minutes = len(data)/(60*fs)
seconds = (len(data)/fs) % 60 
print(f'Recording duration: {int(minutes)} minutes and {int(seconds)} seconds')

trigger = data.iloc[:, -1].to_numpy(dtype=np.float64)
print(np.unique(trigger))

eeg = data.iloc[:, 0:8].to_numpy(dtype=np.float64)

raw_data = convert_to_mne(eeg, trigger, rescale=rescale, fs=fs, chs=chs, recompute=False)

         Fz         C3         Cz        C4        Pz         PO7         Oz  \
0  352731.5  85744.740 -412950.70  315392.7  135223.8   53872.540  419977.90   
1  461437.1  -1251.323 -487115.70  500732.4  132128.6  -64413.440  633890.60   
2  459271.8 -63242.630 -235110.10  495226.8  123411.7 -170184.300  659301.10   
3  456777.0 -32771.390   21885.24  322125.6  248388.3 -139720.200  435231.90   
4  432940.6   6474.501   58251.24  114632.7  422354.8   -2040.408   66089.82   

          PO8  Trigger  
0  233148.900      0.0  
1  185265.600      0.0  
2   63492.360      0.0  
3     959.246      0.0  
4  -46488.850      0.0  
Recording duration: 6 minutes and 10 seconds
[0.]
Creating RawArray with float64 data, n_channels=8, n_times=92509
    Range : 0 ... 92508 =      0.000 ...   370.032 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=92509
    Range : 0 ... 92508 =      0.000 ...   370.032 secs
Ready.


In [21]:
# Compute PSD
Pxx = raw_data.compute_psd(fmin=0, fmax=fs/2)
Pxx.plot()
plt.show()

Effective window size : 8.192 (s)


In [26]:
filtered = raw_data.copy() # the method filters the signal in-place, so this time I
                      # want to preserve the original signal and filter just a
                      # temporary copy of it

# remove power line noise
filtered.notch_filter(50) 
filtered.notch_filter(60) 
# Apply band-pass filtering
filtered.filter(1,30) 

pxx_filt = filtered.compute_psd(fmin=0, fmax=50)
pxx_filt.plot()
plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband e

In [36]:
filtered.plot()
plt.show()

Channels marked as bad:
none


### Channels Interpolation
Previously marked bad channels can be interpolated

In [32]:
## Interpolate bad channels
interp = filtered.copy()

# Mark the bad channels in the raw data
print(interp.info['bads'])

interp.interpolate_bads()
interp.plot()
plt.show()

[]
Setting channel interpolation method to {'eeg': 'spline'}.


C:\Users\Gianni\AppData\Local\Temp\ipykernel_10712\2769900535.py:7: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  interp.interpolate_bads()


Channels marked as bad:
none


### Cleaning data using ASR
Full example available: https://github.com/DiGyt/asrpy/blob/main/example.ipynb

In [33]:
## take the first 30 seconds of the data
training_data = interp.copy()
training_data.crop(tmin=5, tmax=65)
training_data.plot()
plt.show()

Channels marked as bad:
none


In [35]:
import asrpy

asr = asrpy.ASR(sfreq=fs, cutoff=15)
asr.fit(training_data)
cleaned = asr.transform(interp)
cleaned.plot()
plt.show()

Channels marked as bad:
none
